Este script realiza una comparación entre varios modelos de regresión para predecir la mortalidad en función de diversas variables climáticas, como las temperaturas extremas, los días consecutivos de calor y humedad, entre otras. Utiliza datos previamente modificados y limpios para entrenar y probar diferentes algoritmos de machine learning y series temporales.

El objetivo principal del script es evaluar el rendimiento de los modelos mediante métricas como el Mean Squared Error (MSE) y el Coeficiente de Determinación (R²), con el fin de identificar qué modelo ofrece la mejor predicción. Los modelos utilizados incluyen:

Regresión de Poisson.
.-ARIMA para series temporales.
.-Random Forest.
.-Lasso con Grid Search.
.-Árbol de decisión.
.-Redes neuronales.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import PoissonRegressor, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os

# Ignorar advertencias de ciertas librerías
warnings.filterwarnings("ignore", category=UserWarning)  # Ignorar advertencias del usuario
warnings.filterwarnings("ignore", category=FutureWarning)  # Ignorar advertencias futuras
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # Ignorar advertencias de convergencia
warnings.filterwarnings("ignore", category=ValueWarning)  # Ignorar advertencias de valor


# Cargar el dataset desde la carpeta data
file_path = os.path.join('data', 'dataset_modificado.csv')
df = pd.read_csv(file_path)

# Variables independientes y dependiente
variables_independientes = [
    'temp_min_max', 'temp_max_max', 'humedad_max',
    'dias_consecutivos_tmin_22', 'dias_consecutivos_tmin_25',
    'dias_consecutivos_tmin_28', 'dias_consecutivos_tmax_35',
    'dias_consecutivos_tmax_38', 'dias_consecutivos_tmax_41',
    'dias_consecutivos_humedad_70', 'dias_consecutivos_humedad_80',
    'dias_consecutivos_humedad_90', 'dias'
]
variable_dependiente = 'mortalidad'

# Eliminar filas con valores nulos
df_clean = df.dropna(subset=variables_independientes + [variable_dependiente])

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df_clean[variables_independientes]
y = df_clean[variable_dependiente]

# Estandarizar las variables independientes
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Diccionario para almacenar los resultados
resultados_modelos = {}

### 1. Modelo de Poisson
poisson_model = PoissonRegressor()
poisson_model.fit(X_train, y_train)
y_pred_poisson = poisson_model.predict(X_test)

# Métricas para Poisson
mse_poisson = mean_squared_error(y_test, y_pred_poisson)
r2_poisson = r2_score(y_test, y_pred_poisson)
resultados_modelos['Poisson'] = {'MSE': mse_poisson, 'R²': r2_poisson}

### 2. Modelo ARIMA (serie temporal)
arima_model = ARIMA(y_train, order=(1, 1, 1))
arima_fit = arima_model.fit()
y_pred_arima = arima_fit.forecast(steps=len(y_test))

# Métricas para ARIMA
mse_arima = mean_squared_error(y_test, y_pred_arima)
r2_arima = r2_score(y_test, y_pred_arima)
resultados_modelos['ARIMA'] = {'MSE': mse_arima, 'R²': r2_arima}

### 3. Random Forest
random_forest_model = RandomForestRegressor()
random_forest_model.fit(X_train, y_train)
y_pred_rf = random_forest_model.predict(X_test)

# Métricas para Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
resultados_modelos['Random Forest'] = {'MSE': mse_rf, 'R²': r2_rf}

### 4. Lasso con Grid Search
lasso = Lasso()
param_grid_lasso = {'alpha': [0.01, 0.1, 1, 10, 100]}
grid_search_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5)
grid_search_lasso.fit(X_train, y_train)
y_pred_lasso = grid_search_lasso.predict(X_test)

# Métricas para Lasso
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
resultados_modelos['Lasso'] = {'MSE': mse_lasso, 'R²': r2_lasso}

### 5. Árbol de Decisión
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

# Métricas para Árbol de Decisión
mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)
resultados_modelos['Decision Tree'] = {'MSE': mse_tree, 'R²': r2_tree}

### 6. Redes Neuronales
mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42, max_iter=1000)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

# Métricas para Redes Neuronales
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)
resultados_modelos['Red Neuronal'] = {'MSE': mse_mlp, 'R²': r2_mlp}

# Imprimir los resultados para cada modelo
print("Resultados de los Modelos:")
for modelo, metricas in resultados_modelos.items():
    print(f"{modelo}: MSE={metricas['MSE']:.4f}, R²={metricas['R²']:.4f}")

# Guardar resultados en un archivo CSV
resultados_df = pd.DataFrame(resultados_modelos).T
resultados_df.to_csv(os.path.join('data', 'resultados_modelos_comparacion.csv'), index=True)

# Visualizar los resultados en un gráfico de barras
resultados_df.plot(kind='bar', figsize=(10, 6))
plt.title('Comparación de Modelos - MSE y R²')
plt.ylabel('Valor')
plt.xticks(rotation=45)
plt.tight_layout()

# Guardar la gráfica
output_path = os.path.join('data', 'graficos', 'comparacion_modelos.png')
plt.savefig(output_path)
plt.show()

print(f"Resultados guardados en {output_path}")
